In [1]:
import torch
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch import nn
import numpy as np
import torch.nn.functional as F

from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data.sampler import  SubsetRandomSampler  #for validation test

In [2]:
# Check that MPS is available, if not, check if CUDA is available, if not, use CPU
if not torch.backends.mps.is_available():
    if torch.cuda.is_available():
        device = torch.device("cuda:1")
    else:
        device = torch.device("cpu")
else:
    device = torch.device("mps")

(i)knowledge-oblivious–the attacker shall have no knowledge of the target
model’s parameters/structures, nor the original training datasets, 

(ii) cleanlabel–the attacker shall not be able to control the labeling process, and

(iii)clean-test–test-time instances shall not be required to be modified using added
adversarial perturbations for attacking effectiveness

https://www.ecva.net/papers/eccv_2020/papers_ECCV/papers/123720137.pdf

Make sure that you do not reuse the data for training, testing and the attack/defence. - prof

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(0.2859, 0.3530)
                               ])

trainset = datasets.FashionMNIST('MNIST_data/', download = True, train = True, transform = transform)
testset = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)

#Preparing for validaion test
indices = list(range(len(trainset)))
np.random.shuffle(indices)
#to get 20% of the train set
split = int(np.floor(0.2 * len(trainset)))
train_sample = SubsetRandomSampler(indices[:split])
valid_sample = SubsetRandomSampler(indices[split:])

#Data Loader
trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sample, batch_size=64)
validloader = torch.utils.data.DataLoader(trainset, sampler=valid_sample, batch_size=64)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

In [4]:
# define some helper functions
def get_item(preds, labels):
    """function that returns the accuracy of our architecture"""
    return preds.argmax(dim=1).eq(labels).sum().item()

@torch.no_grad() # turn off gradients during inference for memory effieciency
def get_all_preds(network, dataloader):
    """function to return the number of correct predictions across data set"""
    all_preds = torch.tensor([])
    model = network
    for batch in dataloader:
        images, labels = batch
        preds = model(images) # get preds
        all_preds = torch.cat((all_preds, preds), dim=0) # join along existing axis
        
    return all_preds

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(15, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [5]:
class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels=6, kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features= 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 60)
        self.out = nn.Linear(in_features= 60, out_features = 10)
        
        
    def forward(self, tensor):
        
        # hidden layer 1
        tensor = self.conv1(tensor)
        tensor = F.relu(tensor)
        tensor = F.max_pool2d(tensor, kernel_size = 2, stride= 2)
        
        # hidden layer 2
        
        tensor = self.conv2(tensor)
        tensor = F.relu(tensor)
        tensor = F.max_pool2d(tensor, kernel_size = 2, stride = 2)
        
        #hidden layer 3
        
        tensor = tensor.reshape(-1, 12 * 4* 4)
        tensor = self.fc1(tensor)
        tensor = F.relu(tensor)
        
        #hidden layer 4
        
        tensor = self.fc2(tensor)
        tensor = F.relu(tensor)
        
        #output layer
        
        tensor = self.out(tensor)
        
        return tensor

In [6]:
cnn_model = FashionCNN().to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr = 0.005)
criterion = nn.CrossEntropyLoss()
print(cnn_model) # print model structure

FashionCNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


In [7]:
train_losses, valid_losses = [], []
epochs = 10

# Lists for knowing classwise accuracy
predictions_list, labels_list = [], []

for e in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    # Flatten Fashion-MNIST images into a 784 long vector
    images = images.to(device)
    labels = labels.to(device)
    # Training pass
    optimizer.zero_grad()
    
    output = cnn_model.forward(images)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
  else:
    valid_loss, correct, total = 0, 0, 0
    
    # Turn off gradients for validation, saves memory and computation
    with torch.no_grad():
      # Set the model to evaluation mode
      cnn_model.eval()
      
      # Validation pass
      for images, labels in validloader:
        images = images.to(device)
        labels = labels.to(device)
        log_ps = cnn_model(images)
        valid_loss += criterion(log_ps, labels)
        
        predictions = torch.max(log_ps, 1)[1].to(device)
        predictions_list.append(predictions)
        correct += (predictions == labels).sum()
        total += len(labels)
    
    accuracy = correct * 100 / total
    train_losses.append(running_loss/len(trainloader))
    valid_losses.append(valid_loss/len(validloader))
    cnn_model.train()
    
    
    print("Epoch: {}/{}..".format(e+1, epochs),
          "Training loss: {:.3f}..".format(running_loss/len(trainloader)),
          "Validation loss: {:.3f}..".format(valid_loss/len(validloader)),
          "Validation Accuracy: {:.3f}".format(accuracy))
    torch.save(cnn_model.state_dict(), 'model.pt')

Epoch: 1/10.. Training loss: 0.813.. Validation loss: 0.576.. Validation Accuracy: 77.981
Epoch: 2/10.. Training loss: 0.516.. Validation loss: 0.486.. Validation Accuracy: 81.310
Epoch: 3/10.. Training loss: 0.445.. Validation loss: 0.433.. Validation Accuracy: 83.983
Epoch: 4/10.. Training loss: 0.392.. Validation loss: 0.456.. Validation Accuracy: 83.331
Epoch: 5/10.. Training loss: 0.362.. Validation loss: 0.422.. Validation Accuracy: 84.785
Epoch: 6/10.. Training loss: 0.345.. Validation loss: 0.393.. Validation Accuracy: 86.396
Epoch: 7/10.. Training loss: 0.313.. Validation loss: 0.396.. Validation Accuracy: 85.685
Epoch: 8/10.. Training loss: 0.303.. Validation loss: 0.397.. Validation Accuracy: 86.267
Epoch: 9/10.. Training loss: 0.288.. Validation loss: 0.425.. Validation Accuracy: 85.846
Epoch: 10/10.. Training loss: 0.272.. Validation loss: 0.403.. Validation Accuracy: 85.571


In [8]:
def output_label(label):
    output_mapping = {
                 0: "T-shirt/Top",
                 1: "Trouser",
                 2: "Pullover",
                 3: "Dress",
                 4: "Coat", 
                 5: "Sandal", 
                 6: "Shirt",
                 7: "Sneaker",
                 8: "Bag",
                 9: "Ankle Boot"
                 }
    input = (label.item() if type(label) == torch.Tensor else label)
    return output_mapping[input]

In [9]:
class_correct = [0. for _ in range(10)]
total_correct = [0. for _ in range(10)]

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        test = Variable(images)
        outputs = cnn_model(test)
        predicted = torch.max(outputs, 1)[1]
        c = (predicted == labels).squeeze()
        
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            total_correct[label] += 1
        
for i in range(10):
    print("Accuracy of {}: {:.2f}%".format(output_label(i), class_correct[i] * 100 / total_correct[i]))

Accuracy of T-shirt/Top: 81.20%
Accuracy of Trouser: 91.60%
Accuracy of Pullover: 72.50%
Accuracy of Dress: 90.90%
Accuracy of Coat: 86.50%
Accuracy of Sandal: 94.80%
Accuracy of Shirt: 47.80%
Accuracy of Sneaker: 97.20%
Accuracy of Bag: 96.10%
Accuracy of Ankle Boot: 91.90%
